In [2]:
import xarray as xr 
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

import functions.data_wrangling as dw
import functions.eddy_feedback as ef

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in divide")

# All models together

See this [notebook](/home/links/ct715/eddy_feedback/reproduce/eddy_feedback_param/pamip_efp_maths.ipynb) for the original file.

## Combined datasets (Non-regridded)

- More recent datasets created for SAM index (requiring u=u(lon,lat,...))

In [3]:
data_dir = '/home/links/ct715/data_storage/PAMIP/processed_monthly/combined_ua_epfy_divFy/1.1_pdSST-pdSIC'

ds = {}
# Loop through all .nc files in the directory
for filename in sorted(os.listdir(data_dir)):
    if filename.endswith('.nc'):
        parts = filename.split('_')
        model = parts[0]  # Model name

        key = f'{model}'
        file_path = os.path.join(data_dir, filename)

        # Load dataset and assign model and realization as attributes
        ds[key] = xr.open_dataset(file_path).assign_attrs(model=model)


In [4]:
data_path = '/home/links/ct715/eddy_feedback/reproduce/data/efp'
data_file = os.path.join(data_path, 'efp_NH_SH_mon_new-fn_combined_3x3.csv')

if os.path.exists(data_file):
    df = pd.read_csv(data_file)
else:
    # Initialize lists to store results
    model_list = []
    efp_djf = []   # NH Feedback (DJF)
    efp_jas = []   # SH Feedback (JAS)

    # Loop through each model's dataset
    for key, dataset in ds.items():
        model_name = dataset.attrs.get('model', key)  # Fallback to key if attribute missing

        # Calculate EFP for NH (default) and SH
        nh_feedback = ef.calculate_efp(dataset, data_type='pamip')
        sh_feedback = ef.calculate_efp(dataset, calc_south_hemis=True, data_type='pamip')

        # Append results
        model_list.append(model_name)
        efp_djf.append(nh_feedback)
        efp_jas.append(sh_feedback)

    # Create DataFrame
    pd_data = {
        'Model': model_list,
        'NH Feedback (DJF)': efp_djf,
        'SH Feedback (JAS)': efp_jas
    }

    df = pd.DataFrame(pd_data)
    df = df.sort_values('Model').reset_index(drop=True) # sort by model name
    df.to_csv(data_file, index=False)
    
df

,Model,NH Feedback (DJF),SH Feedback (JAS)
0,CESM2,0.3693,0.3419
1,CNRM-CM6-1,0.2421,0.2082
2,CanESM5,0.4256,0.4618
3,EC-EARTH3,0.2945,0.3135
4,FGOALS-f3-L,0.2354,0.1274
5,HadGEM3-GC31-LL,0.2730,0.2951
6,HadGEM3-GC31-MM,0.3137,0.3002
7,IPSL-CM6A-LR,0.2719,0.1959
8,MIROC6,0.2213,0.3253
9,NorESM2-LM,0.3238,0.3754


## Original saved files (regridded 3x3)

In [5]:
data_dir = '/home/links/ct715/data_storage/PAMIP/processed_monthly/regridded_3x3/1.1_pdSST-pdSIC'

ds = {}
# Loop through all .nc files in the directory
for filename in sorted(os.listdir(data_dir)):
    if filename.endswith('.nc'):
        parts = filename.split('_')
        model = parts[0]  # Model name

        key = f'{model}'
        file_path = os.path.join(data_dir, filename)

        # Load dataset and assign model and realization as attributes
        ds[key] = xr.open_dataset(file_path).assign_attrs(model=model)


In [6]:
data_path = '/home/links/ct715/eddy_feedback/reproduce/data/efp'
data_file = os.path.join(data_path, 'efp_NH_SH_mon_new-fn_regridded_3x3.csv')

if os.path.exists(data_file):
    df = pd.read_csv(data_file)
else:
    # Initialize lists to store results
    model_list = []
    efp_djf = []   # NH Feedback (DJF)
    efp_jas = []   # SH Feedback (JAS)

    # Loop through each model's dataset
    for key, dataset in ds.items():
        model_name = dataset.attrs.get('model', key)  # Fallback to key if attribute missing

        # Calculate EFP for NH (default) and SH
        nh_feedback = ef.calculate_efp(dataset, data_type='pamip')
        sh_feedback = ef.calculate_efp(dataset, calc_south_hemis=True, data_type='pamip')

        # Append results
        model_list.append(model_name)
        efp_djf.append(nh_feedback)
        efp_jas.append(sh_feedback)

    # Create DataFrame
    pd_data = {
        'Model': model_list,
        'NH Feedback (DJF)': efp_djf,
        'SH Feedback (JAS)': efp_jas
    }

    df = pd.DataFrame(pd_data)
    df = df.sort_values('Model').reset_index(drop=True) # sort by model name
    df.to_csv(data_file, index=False)
    
df

,Model,NH Feedback (DJF),SH Feedback (JAS)
0,CESM1-WACCM-SC,0.1421,0.0303
1,CESM2,0.4132,0.3714
2,CNRM-CM6-1,0.2771,0.2492
3,CanESM5,0.4446,0.4808
4,E3SMv1,0.0589,0.0588
5,EC-EARTH3,0.3467,0.3621
6,ECHAM6.3_AWI,0.0144,0.0104
7,FGOALS-f3-L,0.2642,0.1512
8,HadGEM3-GC31-LL,0.3168,0.3412
9,HadGEM3-GC31-MM,0.3757,0.3621
